In [2]:
import numpy as np
import random
import pickle
from simple_custom_taxi_env import SimpleTaxiEnv
import time

In [3]:
np.bool8 = np.bool_

env_config = {
    "fuel_limit": 5000
}
render = False
hyperparameters = {
    "alpha": 0.1,
	"gamma": 0.99,
	"epsilon_start": 1.0, 
 	"epsilon_end": 0.1,
	"decay_rate": 0.9999,
	"episodes": 10000
}

In [8]:

q_table = {}
epsilon = hyperparameters["epsilon_start"]

def get_state(obs):
	if not hasattr(get_state, "passenger_loc"):
		get_state.passenger_loc = None
	if not hasattr(get_state, "destination_loc"):
		get_state.destination_loc = None
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col, stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]	
 # determine which station the taxi is at
	taxi_station = 0
	for i in range(4):
		if taxi_row == stations[i][0] and taxi_col == stations[i][1]:
			taxi_station = i
			break
	
	has_picked_up = False
	if passenger_look == 1 and get_state.destination_loc is None:
		# determine the passenger location
		for i in range(4):
			passenger_loc_north = int( (taxi_row - 1, taxi_col) == stations[i])
			passenger_loc_south = int( (taxi_row + 1, taxi_col) == stations[i])
			passenger_loc_east  = int( (taxi_row, taxi_col + 1) == stations[i])
			passenger_loc_west  = int( (taxi_row, taxi_col - 1) == stations[i])
			passenger_loc_middle  = int( (taxi_row, taxi_col) == stations[i])
			if passenger_loc_north or passenger_loc_south or passenger_loc_east or passenger_loc_west or passenger_loc_middle:
				get_state.passenger_loc = (stations[i][0], stations[i][1])
				break
		if passenger_loc is None or passenger_loc_middle:
			has_picked_up = True
	if has_picked_up:
		get_state.passenger_loc = None
  
	if destination_look == 1 and get_state.destination_loc is None:
		# determine the destination location
		for i in range(4):
			destination_loc_north = int( (taxi_row - 1, taxi_col) == stations[i])
			destination_loc_south = int( (taxi_row + 1, taxi_col) == stations[i])
			destination_loc_east  = int( (taxi_row, taxi_col + 1) == stations[i])
			destination_loc_west  = int( (taxi_row, taxi_col - 1) == stations[i])
			destination_loc_middle  = int( (taxi_row, taxi_col) == stations[i])
			# print (taxi_row, taxi_col, stations[i])
			if destination_loc_north or destination_loc_south or destination_loc_east or destination_loc_west or destination_loc_middle:
				get_state.destination_loc = (stations[i][0], stations[i][1])
				break
		# print (destination_loc)
	
	
	return (int(taxi_row), int(taxi_col), obstacle_north, obstacle_south, obstacle_east, obstacle_west, get_state.passenger_loc, get_state.destination_loc, has_picked_up)
  
def get_action(obs):
	"""
	# Selects the best action using the trained Q-table.
	"""
	if np.random.uniform(0, 1) < epsilon:
		action = np.random.choice(action_nums)  # Random action
	else:
		action = np.argmax(q_table[get_state(obs)])  # Greedy action
	return action

env = SimpleTaxiEnv(**env_config)
action_nums = 6
rewards_per_episode = []

obs, _ = env.reset()
total_reward = 0
done = False
step_count = 0
stations = [(0, 0), (0, 4), (4, 0), (4,4)]

taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs

if render:
	env.render_env((taxi_row, taxi_col),
					action=None, step=step_count, fuel=env.current_fuel)
	time.sleep(0.5)
 

for episode in range(hyperparameters["episodes"]):
	get_state.passenger_loc, get_state.destination_loc = None, None
	obs, _ = env.reset()
	done = False
	total_reward = 0
	step_count = 0
	passenger_loc = None
	destination = None
	state = get_state(obs)
	while not done:	
		if state not in q_table:
			q_table[state] = np.zeros(action_nums)
   
		action = get_action(obs)

		next_obs, reward, done, _ = env.step(action)
		if done:
			if reward < 40:
				done = False
			else:
				pass
				# print (reward)
				# print ('done', reward, action, state, get_state(next_obs), next_obs[-2:])
		# print (next_obs)
   
		next_state = get_state(next_obs)
		# shaped_reward = reward-1
		# if next_state[-1]:
		# 	shaped_reward += 100
		# elif done and reward > 40:
		# 	shaped_reward += 100
   
		total_reward += reward
		if next_obs not in q_table:
			# print (next_obs)
			q_table[next_state] = np.zeros(action_nums)
		q_table[state][action] += hyperparameters["alpha"] * (reward + hyperparameters["gamma"] * np.max(q_table[next_state]) - q_table[state][action])
		
		step_count += 1
		obs = next_obs
		state = next_state
		total_reward += reward

		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look,destination_look = obs
	
		if render:
			env.render_env((taxi_row, taxi_col),
							action=action, step=step_count, fuel=env.current_fuel)
	# print (step_count)
	rewards_per_episode.append(total_reward)
	epsilon = max(hyperparameters["epsilon_end"], epsilon * hyperparameters["decay_rate"])
	if (episode + 1) % 20 == 0:
		avg_reward = np.mean(rewards_per_episode[-20:])
		print(f'Episode {episode + 1}/{hyperparameters["episodes"]}, Avg Reward: {avg_reward:.4f}, Epsilon: {epsilon:.3f}')
		# print ([np.argmax(i) for i in q_table.values()])
		print (len(q_table))

Episode 20/10000, Avg Reward: -146618.0600, Epsilon: 0.998
146
Episode 40/10000, Avg Reward: -150002.1200, Epsilon: 0.996
151
Episode 60/10000, Avg Reward: -154064.4700, Epsilon: 0.994
151
Episode 80/10000, Avg Reward: -138112.5800, Epsilon: 0.992
151
Episode 100/10000, Avg Reward: -181612.8700, Epsilon: 0.990
153
Episode 120/10000, Avg Reward: -205935.0300, Epsilon: 0.988
153
Episode 140/10000, Avg Reward: -123030.7200, Epsilon: 0.986
154
Episode 160/10000, Avg Reward: -107743.9600, Epsilon: 0.984
154
Episode 180/10000, Avg Reward: -219559.0500, Epsilon: 0.982
155
Episode 200/10000, Avg Reward: -123854.9400, Epsilon: 0.980
155
Episode 220/10000, Avg Reward: -145847.2300, Epsilon: 0.978
155
Episode 240/10000, Avg Reward: -140428.0600, Epsilon: 0.976
155
Episode 260/10000, Avg Reward: -200379.3800, Epsilon: 0.974
155
Episode 280/10000, Avg Reward: -121319.2500, Epsilon: 0.972
155
Episode 300/10000, Avg Reward: -152175.7800, Epsilon: 0.970
155
Episode 320/10000, Avg Reward: -142925.1400,

KeyboardInterrupt: 